In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import data_analysis_tools_battery_3 as tools
import re
import random

## Config

In [ ]:
OVERRIDE_OLD_FIXED = False
TEST = False

GENERATE_REPORTS = True
SHOW_REPORTS = True
CUT_RESULTS_TO = None
DISABLE_NEW_REPORT = False
INTERVALAR_PLOT = True

In [ ]:
base_path = f"{os.getcwd()}/../reports/"

tests = [
	# f"{base_path}tests/",
	# f"{base_path}tests/NEAT/speciation/",
	# f"{base_path}tests/NEAT/history_size_100/NB4/",
	# f"{base_path}tests/NEAT/history_size_100/B1/",
	# f"{base_path}tests/NEAT/history_size_100/NB4-path-grading/",
	# f"{base_path}tests/NEAT/speciation/saved_logs/no_speciation",
	# f"{base_path}tests/NEAT/speciation/saved_logs/speciated",
	# f"{base_path}tests/NEAT/battery_3/take_1/mov/",
	# f"{base_path}tests/NEAT/battery_3/take_1/updates/",
	# f"{base_path}tests/NEAT/battery_3/_movement_captures/",
]

battery_1 = [
	# f"{base_path}battery_1/",
	### f"{base_path}battery_1_baseline_take_2/",
	# f"{base_path}battery_1_baseline_take_2_new_reports/"
]

battery_2 = [
	# f"{base_path}battery_2/B1/",
	# f"{base_path}battery_2/B1-path-grading-hack/",
	# f"{base_path}battery_2/B1-path-grading-simple/",
	# f"{base_path}battery_2/B2/",
	# f"{base_path}battery_2/B3/",
	# f"{base_path}battery_2/NB4/",
	# f"{base_path}battery_2/NB4-path-grading/",
]

battery_3= [
	# f"{base_path}battery_3_config_baseline/",
	# f"{base_path}battery_3/tests/",
	# f"{base_path}battery_3/take_1/",
	# f"{base_path}battery_3/take_1_neat/",
	f"{base_path}battery_3/take_2_neat/",
	# f"{base_path}battery_3/take_3_neat/",
]

selected_paths = battery_1 + battery_2 + battery_3 +  tests

In [ ]:
def prepare_data_fixed(path, selected_runs):
	for selected_run in selected_runs:
		this_path = f"{path}{selected_run}.data"
		if not os.path.exists(this_path + "_fixed") or OVERRIDE_OLD_FIXED:
			if not os.path.exists(this_path) or OVERRIDE_OLD_FIXED:
				#join data if necessary
				tools.join_data(this_path)
			tools.quote_gene(this_path)

def get_availiable_runs(filenames_list):
	selected_runs_set = set()
	for filename in filenames_list:
		found = re.findall("[0-9]+", filename)
		if len(found) > 0:
			selected_runs_set.add(found[0])
	return list(selected_runs_set)

def process_all_in_path(path):
	in_path_df_dict = dict()
	selected_runs = get_availiable_runs(os.listdir(path))
	prepare_data_fixed(path, selected_runs)
	errors = list()

	for run in selected_runs:
		print(path, run)
		run_dict = dict()
		run_dict["run_id"] = run
		run_dict["path"] = path
		with open(f"{path}{run}.conf", 'r') as f:
			for line in f.readlines():
				if tools.ANN_MODE_STRING in line:
					this_mode = line.replace(tools.ANN_MODE_STRING, "")
					run_dict["mode"] = this_mode
				elif tools.ANN_LAYERS_STRING in line:
					this_layers = line.replace(tools.ANN_LAYERS_STRING, "")
					run_dict["layers"] = this_layers
				elif tools.ANN_FITNESS_STRING in line:
					this_fitness = line.replace(tools.ANN_FITNESS_STRING, "")
					run_dict["fitness"] = this_fitness
				elif tools.ANN_NEAT_ENABLED_STRING in line:
					this_neat_enable = line.replace(tools.ANN_NEAT_ENABLED_STRING, "").strip()
					if this_neat_enable.lower() == "true":
						run_dict["neat_enabled"] = True
					else:
						run_dict["neat_enabled"] = False

		run_dict["df"], load_errors = tools.load_dataframe(path, run, neat_enabled = run_dict["neat_enabled"])
		in_path_df_dict[run] = run_dict
		errors.extend(load_errors)
		if TEST == True:
			return in_path_df_dict, errors

	return in_path_df_dict, errors


## Do it

In [ ]:
errors = list()
paths_dict = dict()
for path in selected_paths:
	paths_dict[path], load_errors = process_all_in_path(path)
	errors.extend(load_errors)
	if TEST == True:
		break

# run count
run_count = 0
for index, path_dict in paths_dict.items():
	for _, run_dict in path_dict.items():
		run_count = run_count + 1
		if GENERATE_REPORTS:
			errors.extend(tools.generate_run_report_from_dict(run_dict, show = SHOW_REPORTS, CUT_RESULTS_TO = CUT_RESULTS_TO, disable_new_report = DISABLE_NEW_REPORT, intervalar_plot = INTERVALAR_PLOT))

print("Path count: ", len(paths_dict.keys()))
print("Run count: ", run_count)
print("Errors:\n" + str(errors))